In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd
from functools import reduce
import os

# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
from cryptography.fernet import Fernet
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False)

In [4]:
# generate and store phone numbers
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10

# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

#shuffle phone number lists
random.shuffle(gojek_phone_numbers)
random.shuffle(grab_phone_numbers)

# write phone numbers to csv file
phone_number_storage_manager(gojek_phone_numbers, grab_phone_numbers)

# Define classes for the necessary for the algorithm

In [5]:
# class to generate clients' private secret
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q]
        
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    def generate_random_number(self, size):
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation
            if (result == 1):
                return False
            
        return True 

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False)

                


    
        

# Create psi client class

In [6]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1, fernet_key):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        # for asymmetric encryption
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p
        self.factors_pminus1 = factors_pminus1
        
        # for symmetric encryption
        self.fernet_key = fernet_key
        self.f = Fernet(fernet_key)
        
        self.my_hashed_set = None
        self.my_self_encrypted_set = None
        self.my_encrypted_set = None
        self.other_party_encrypted_set = None
        self.common_values = None
        
        # create datafile for communication with another party
        # only need to share self_encrypted_values and other_party_encrypted_values
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        self.name = name
        self.filename = name + "_data_v1.1.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.1.csv"
        
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element):     
        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() #sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
        return hash_int
    
    def modular_exponentation(self, element):
        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        
        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        
        if (is_other_party):
            decrypted_other_party_set = self.receive_data("my_self_encrypted_set")
            other_party_set_int = []
            for element_string in decrypted_other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        else:
            set_to_encrypt = self.my_hashed_set
        
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
                    
        
    def get_intersection(self):
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set") # encrypted refers to asymmetric_crypto

        my_encrypted_set_int = []
        for element in my_encrypted_set:
            my_encrypted_set_int.append(int(element))
        self.my_encrypted_set = my_encrypted_set_int
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        for index in index_of_common_values:
            self.common_values.append(self.my_set[index])
        self.my_dict["common_values"] = self.common_values
        self.send_data(self.common_values, "common_values")
    
    def encrypt_data(self, plaintext):
        endian = "big"
        element = plaintext.to_bytes(128, endian) # 1024 bits == 128 bytes
        cipher_text = self.f.encrypt(element)
        return cipher_text
    
    def decrypt_data(self, ciphertext):
        ciphertext_bytes = ciphertext.encode('utf-8')[2:-1] # convert from string back to bytes
        endian = "big"
        element_in_bytes = self.f.decrypt(ciphertext_bytes)
        plaintext = int.from_bytes(element_in_bytes, endian)
        return plaintext

    def send_data(self, data_to_send, column_name):
        # send data means writing to file
        encrypted_data_to_send = []
        for element in data_to_send:
            encrypted_element = self.encrypt_data(element)
            encrypted_data_to_send.append(encrypted_element)
        self.my_dict[column_name] = encrypted_data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file
        encrypted_data = self.get_other_party_data()[column_name].to_list()
        decrypted_data = []
        for element in encrypted_data:
            if type(element) is float:
                continue
            decrypted_element = self.decrypt_data(element)
            decrypted_data.append(decrypted_element)
        print(f"number of elements in {self.other_party_name};{column_name}: {len(decrypted_data)}.")
        return decrypted_data
            
    def get_my_data(self):
        return pd.read_csv(self.filename)
    
    def get_other_party_data(self):
        return pd.read_csv(self.other_party_filename)
    
    


# Initialize context

In [7]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

# create key for symmetric key cryptography
fernet_key = Fernet.generate_key()

# create public parameters required for psi
number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_phone_numbers, p, factors_pminus1, fernet_key)
gojek = Client("gojek", "grab", key_size, gojek_phone_numbers, p, factors_pminus1, fernet_key)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_9136/525289407.py:47: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

## Step 1: Hash phone numbers

In [8]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Clients' status after step 1
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [9]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,6266522927071483469142230791564284094912332188...,1751659343934249496930093781941606327015039027...
1,4464107689706471902374892139671036333433126606...,5930688136427591374346485570270456146366904370...
2,2953623139145187618071647113313426330528576167...,8096241512554496249604175498222045756542415455...
3,4271190063387899149690355220761209149002412790...,1017141269956754882131908776753968058318200171...
4,1269677790173295089009790716971253424972418504...,3864964095203405700528472210556810964594978309...
...,...,...
86,NaN,1105794065386265187900241838908783069718325271...
87,NaN,3683371775181761031500661295222456997304332645...
88,NaN,3062635494707074836246422469968877214990302591...
89,NaN,5011913842513548422134267800545138946266586949...


## Step 2: encrypt hashed set with own private key

In [10]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_9136/525289407.py:47: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


### Clients' status after step 2 (value of variables in client)

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,6266522927071483469142230791564284094912332188...,1751659343934249496930093781941606327015039027...,9806539572105544239792553962012412568042870553...,6773191962594436426860740795678032593450231537...
1,4464107689706471902374892139671036333433126606...,5930688136427591374346485570270456146366904370...,3781133013163674895508584304598494057434060512...,8804515776459010685854883015394722710825565763...
2,2953623139145187618071647113313426330528576167...,8096241512554496249604175498222045756542415455...,6644895312449464959974714917817751721492461896...,3004433137150398396470100641957113658094678865...
3,4271190063387899149690355220761209149002412790...,1017141269956754882131908776753968058318200171...,1131719771173242106618372476480957079514425326...,1352193231370925340396918306404787587247165082...
4,1269677790173295089009790716971253424972418504...,3864964095203405700528472210556810964594978309...,1083553748494361727426443988574111168881821957...,8668386688678589609297918987430360603625898351...
...,...,...,...,...
86,NaN,1105794065386265187900241838908783069718325271...,NaN,2712404082266171193350697309065278280051176091...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,8981791643925222166499690215903654186571990911...
88,NaN,3062635494707074836246422469968877214990302591...,NaN,1037582850733970933219810748434017500761613669...
89,NaN,5011913842513548422134267800545138946266586949...,NaN,9555514635236807977711494486901487315461243779...


### Files' status after step 2 (value of variables in file - clients' variables encrypted with Fernet)

In [12]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoThuVD7nbyi_fWhZqsi6hHly4a5GBHV1NKgZ...,NaN,NaN
1,b'gAAAAABhoThubIJOFhl8K07u7vcfsFp2_zLqA3eUrP-z...,NaN,NaN
2,b'gAAAAABhoThuEB4xURnGxCSaeKT2uefSG4QOg3xcFY5N...,NaN,NaN
3,b'gAAAAABhoThumgp9q2N49rpiF0a16Bh9iaBTyDgHK87Y...,NaN,NaN
4,b'gAAAAABhoThui_M0YwaQFrylUy6_tWspyUfyI7cLDeeC...,NaN,NaN
...,...,...,...
56,b'gAAAAABhoThuVSTsIF5x2uRNusRP5fF4fbSV7lr9SnA8...,NaN,NaN
57,b'gAAAAABhoThu_Y3tq8rLcbLzIpgUMyHrfPZguyhhXvK_...,NaN,NaN
58,b'gAAAAABhoThu8p5l1TnZUO21aS30i4WhNS5Un3PPUfhT...,NaN,NaN
59,b'gAAAAABhoThuKOv4fWHLIfYumvrbv-o9Y7BeLi54Rigr...,NaN,NaN


In [13]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoThujbPDaA2OhYBk1RSY9zNIAFThdZWngPqF...,NaN,NaN
1,b'gAAAAABhoThuhG9kyqePRTtQCj5LAXQvbC3geI51fsj4...,NaN,NaN
2,b'gAAAAABhoThuYXk6rOoN33BVjRsW0-ufEkRlfQMrQrwV...,NaN,NaN
3,b'gAAAAABhoThuQIR47RsYMZNcIgGLnqt1NtcJd151d3C5...,NaN,NaN
4,b'gAAAAABhoThukFYhCyaTNwwp3pxe6Bqi6skQt0x3eTi2...,NaN,NaN
...,...,...,...
86,b'gAAAAABhoThuj-dd8jt0NHkrQKILJCcg5AtinaF4y66x...,NaN,NaN
87,b'gAAAAABhoThuWLl1LKAH-hgcmlwMw-2QnRoA1FYXUm0Z...,NaN,NaN
88,b'gAAAAABhoThuB76x0IeFJgDyd9NTacOSE07CY_XzPkxH...,NaN,NaN
89,b'gAAAAABhoThuZmt3vF3Y4QiWy-iqCQGElfIAgqf5x5e3...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key


In [14]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 61.


C:\Users\jiaji\AppData\Local\Temp/ipykernel_9136/525289407.py:47: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


number of elements in grab;my_self_encrypted_set: 91.


### Clients' status after step 3

In [15]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,6266522927071483469142230791564284094912332188...,1751659343934249496930093781941606327015039027...,9806539572105544239792553962012412568042870553...,6773191962594436426860740795678032593450231537...,9833885446737399481805887149257310830703291113...,1187492936186447397647201305800360819361802452...
1,4464107689706471902374892139671036333433126606...,5930688136427591374346485570270456146366904370...,3781133013163674895508584304598494057434060512...,8804515776459010685854883015394722710825565763...,4373752830939318225791808445642697952882876364...,8985989641865181228584451284577728252015426643...
2,2953623139145187618071647113313426330528576167...,8096241512554496249604175498222045756542415455...,6644895312449464959974714917817751721492461896...,3004433137150398396470100641957113658094678865...,4788057368962616454392685136139491518182033902...,1201869110185093686389517960533441968074195763...
3,4271190063387899149690355220761209149002412790...,1017141269956754882131908776753968058318200171...,1131719771173242106618372476480957079514425326...,1352193231370925340396918306404787587247165082...,5150368151222151123179608191449730789033183820...,8983007595539318695127288248976845299682165648...
4,1269677790173295089009790716971253424972418504...,3864964095203405700528472210556810964594978309...,1083553748494361727426443988574111168881821957...,8668386688678589609297918987430360603625898351...,1021181929144664103207318197009581208858950472...,9834031398099817354822001747788274908974313815...
...,...,...,...,...,...,...
86,NaN,1105794065386265187900241838908783069718325271...,NaN,2712404082266171193350697309065278280051176091...,NaN,8748468908737766788378042416883984604687344076...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,8981791643925222166499690215903654186571990911...,NaN,1442131364880874739815462291233295910984639261...
88,NaN,3062635494707074836246422469968877214990302591...,NaN,1037582850733970933219810748434017500761613669...,NaN,1234410188056034432384644391342248623239863743...
89,NaN,5011913842513548422134267800545138946266586949...,NaN,9555514635236807977711494486901487315461243779...,NaN,2910038334979038381020012893926960298573065899...


### Files' status after step 3

In [16]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek



gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoThuVD7nbyi_fWhZqsi6hHly4a5GBHV1NKgZ...,b'gAAAAABhoThvGh8TCtlJAJQBWObgnXnKFpEozl6_AtW5...,NaN
1,b'gAAAAABhoThubIJOFhl8K07u7vcfsFp2_zLqA3eUrP-z...,b'gAAAAABhoThvkXhlOpKHOknPbxUtIdK_-EOhuEOim172...,NaN
2,b'gAAAAABhoThuEB4xURnGxCSaeKT2uefSG4QOg3xcFY5N...,b'gAAAAABhoThvEZrAft__j5iExpxUJVhGePhawfLs1fKR...,NaN
3,b'gAAAAABhoThumgp9q2N49rpiF0a16Bh9iaBTyDgHK87Y...,b'gAAAAABhoThvrjXjIwELrfgaNd-px5nV6cpzUGgxzGQn...,NaN
4,b'gAAAAABhoThui_M0YwaQFrylUy6_tWspyUfyI7cLDeeC...,b'gAAAAABhoThvjrVfguBrZTXI6a3gF3LT4fuQen5XVDEG...,NaN
...,...,...,...
86,NaN,b'gAAAAABhoThvLzw7g0bjLkzHm_aHlhPeiTtwCFIqG6O6...,NaN
87,NaN,b'gAAAAABhoThvijAFUbObmPPWsWKwSaajKnxbeFX6jiEC...,NaN
88,NaN,b'gAAAAABhoThvhIei35vjxgem1xMmfr5T9GRU_I0rQ-WR...,NaN
89,NaN,b'gAAAAABhoThvH4p93TDG7yazaoLiT2O9beQBWLYGGsUw...,NaN


In [17]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoThujbPDaA2OhYBk1RSY9zNIAFThdZWngPqF...,b'gAAAAABhoThu4VIO1BmpPsIxB3wV7vKl0Mj8kSdWBEXz...,NaN
1,b'gAAAAABhoThuhG9kyqePRTtQCj5LAXQvbC3geI51fsj4...,b'gAAAAABhoThutbqOAIYcE7kaVabdmNpqANe6d2yjdB32...,NaN
2,b'gAAAAABhoThuYXk6rOoN33BVjRsW0-ufEkRlfQMrQrwV...,b'gAAAAABhoThuQTxNKBQ4CmSPdjbQpGXtaB9dbWTQAX-Z...,NaN
3,b'gAAAAABhoThuQIR47RsYMZNcIgGLnqt1NtcJd151d3C5...,b'gAAAAABhoThuIPfwSnY80Po7EgoZciIpgq3MR5UQ2pkh...,NaN
4,b'gAAAAABhoThukFYhCyaTNwwp3pxe6Bqi6skQt0x3eTi2...,b'gAAAAABhoThu9WnAYgI8_OhESSrgsLA_Mm5MilX12jC9...,NaN
...,...,...,...
86,b'gAAAAABhoThuj-dd8jt0NHkrQKILJCcg5AtinaF4y66x...,NaN,NaN
87,b'gAAAAABhoThuWLl1LKAH-hgcmlwMw-2QnRoA1FYXUm0Z...,NaN,NaN
88,b'gAAAAABhoThuB76x0IeFJgDyd9NTacOSE07CY_XzPkxH...,NaN,NaN
89,b'gAAAAABhoThuZmt3vF3Y4QiWy-iqCQGElfIAgqf5x5e3...,NaN,NaN


## Step 4: find intersection


In [18]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 91.
number of elements in grab;other_party_encrypted_set: 61.


### Clients' status after step 4:

In [19]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,6266522927071483469142230791564284094912332188...,1751659343934249496930093781941606327015039027...,9806539572105544239792553962012412568042870553...,6773191962594436426860740795678032593450231537...,9833885446737399481805887149257310830703291113...,1187492936186447397647201305800360819361802452...,89312048.0,89312048.0
1,4464107689706471902374892139671036333433126606...,5930688136427591374346485570270456146366904370...,3781133013163674895508584304598494057434060512...,8804515776459010685854883015394722710825565763...,4373752830939318225791808445642697952882876364...,8985989641865181228584451284577728252015426643...,96485172.0,96485172.0
2,2953623139145187618071647113313426330528576167...,8096241512554496249604175498222045756542415455...,6644895312449464959974714917817751721492461896...,3004433137150398396470100641957113658094678865...,4788057368962616454392685136139491518182033902...,1201869110185093686389517960533441968074195763...,81093373.0,81093373.0
3,4271190063387899149690355220761209149002412790...,1017141269956754882131908776753968058318200171...,1131719771173242106618372476480957079514425326...,1352193231370925340396918306404787587247165082...,5150368151222151123179608191449730789033183820...,8983007595539318695127288248976845299682165648...,99173089.0,99173089.0
4,1269677790173295089009790716971253424972418504...,3864964095203405700528472210556810964594978309...,1083553748494361727426443988574111168881821957...,8668386688678589609297918987430360603625898351...,1021181929144664103207318197009581208858950472...,9834031398099817354822001747788274908974313815...,80497694.0,80497694.0
...,...,...,...,...,...,...,...,...
86,NaN,1105794065386265187900241838908783069718325271...,NaN,2712404082266171193350697309065278280051176091...,NaN,8748468908737766788378042416883984604687344076...,NaN,NaN
87,NaN,3683371775181761031500661295222456997304332645...,NaN,8981791643925222166499690215903654186571990911...,NaN,1442131364880874739815462291233295910984639261...,NaN,NaN
88,NaN,3062635494707074836246422469968877214990302591...,NaN,1037582850733970933219810748434017500761613669...,NaN,1234410188056034432384644391342248623239863743...,NaN,NaN
89,NaN,5011913842513548422134267800545138946266586949...,NaN,9555514635236807977711494486901487315461243779...,NaN,2910038334979038381020012893926960298573065899...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [20]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoThuVD7nbyi_fWhZqsi6hHly4a5GBHV1NKgZ...,b'gAAAAABhoThvGh8TCtlJAJQBWObgnXnKFpEozl6_AtW5...,b'gAAAAABhoThvbiO2qVlJbOU0rMoPvCVuk2QqHPpBoaHU...
1,b'gAAAAABhoThubIJOFhl8K07u7vcfsFp2_zLqA3eUrP-z...,b'gAAAAABhoThvkXhlOpKHOknPbxUtIdK_-EOhuEOim172...,b'gAAAAABhoThvWUmeFHAEOVCTQ_hjyx2GpIDOAa0CUaVx...
2,b'gAAAAABhoThuEB4xURnGxCSaeKT2uefSG4QOg3xcFY5N...,b'gAAAAABhoThvEZrAft__j5iExpxUJVhGePhawfLs1fKR...,b'gAAAAABhoThv86tzbA5z8emk6Lmq0y-XVW7uus8lmlYt...
3,b'gAAAAABhoThumgp9q2N49rpiF0a16Bh9iaBTyDgHK87Y...,b'gAAAAABhoThvrjXjIwELrfgaNd-px5nV6cpzUGgxzGQn...,b'gAAAAABhoThvGPEp3yYSqYqFArJFXMx9jFoyl3YCzGgv...
4,b'gAAAAABhoThui_M0YwaQFrylUy6_tWspyUfyI7cLDeeC...,b'gAAAAABhoThvjrVfguBrZTXI6a3gF3LT4fuQen5XVDEG...,b'gAAAAABhoThv92vkIOYPO1ZQGGlp38gR10nU03tVdigO...
...,...,...,...
86,NaN,b'gAAAAABhoThvLzw7g0bjLkzHm_aHlhPeiTtwCFIqG6O6...,NaN
87,NaN,b'gAAAAABhoThvijAFUbObmPPWsWKwSaajKnxbeFX6jiEC...,NaN
88,NaN,b'gAAAAABhoThvhIei35vjxgem1xMmfr5T9GRU_I0rQ-WR...,NaN
89,NaN,b'gAAAAABhoThvH4p93TDG7yazaoLiT2O9beQBWLYGGsUw...,NaN


In [21]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoThujbPDaA2OhYBk1RSY9zNIAFThdZWngPqF...,b'gAAAAABhoThu4VIO1BmpPsIxB3wV7vKl0Mj8kSdWBEXz...,b'gAAAAABhoThvrHRCxZvY-70HZ-RGuNKzbV6JPh0YQpyL...
1,b'gAAAAABhoThuhG9kyqePRTtQCj5LAXQvbC3geI51fsj4...,b'gAAAAABhoThutbqOAIYcE7kaVabdmNpqANe6d2yjdB32...,b'gAAAAABhoThvoNqZ2qMtqUkBsdmjl7JSK3oOHSX9wizu...
2,b'gAAAAABhoThuYXk6rOoN33BVjRsW0-ufEkRlfQMrQrwV...,b'gAAAAABhoThuQTxNKBQ4CmSPdjbQpGXtaB9dbWTQAX-Z...,b'gAAAAABhoThvvoAqES9r9o8BtJENj7tl4mq4N21xA1KV...
3,b'gAAAAABhoThuQIR47RsYMZNcIgGLnqt1NtcJd151d3C5...,b'gAAAAABhoThuIPfwSnY80Po7EgoZciIpgq3MR5UQ2pkh...,b'gAAAAABhoThvGkriYL_X3No5LzlcVRKz_kTV642Hwn8g...
4,b'gAAAAABhoThukFYhCyaTNwwp3pxe6Bqi6skQt0x3eTi2...,b'gAAAAABhoThu9WnAYgI8_OhESSrgsLA_Mm5MilX12jC9...,b'gAAAAABhoThvbDv3BE3mZSBKiioDaSSZJoUu0ZN7Aihc...
...,...,...,...
86,b'gAAAAABhoThuj-dd8jt0NHkrQKILJCcg5AtinaF4y66x...,NaN,NaN
87,b'gAAAAABhoThuWLl1LKAH-hgcmlwMw-2QnRoA1FYXUm0Z...,NaN,NaN
88,b'gAAAAABhoThuB76x0IeFJgDyd9NTacOSE07CY_XzPkxH...,NaN,NaN
89,b'gAAAAABhoThuZmt3vF3Y4QiWy-iqCQGElfIAgqf5x5e3...,NaN,NaN


# Check results

In [22]:
# get intersection found by the two parties
gojek_found_intersection = gojek.common_values
grab_found_intersection = grab.common_values


# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,actual,gojek,grab
0,80497694,80497694,80497694
1,81093373,81093373,81093373
2,86915509,86915509,86915509
3,89312048,89312048,89312048
4,94391128,94391128,94391128
5,95521626,95521626,95521626
6,96192082,96192082,96192082
7,96485172,96485172,96485172
8,99173089,99173089,99173089
9,99397525,99397525,99397525
